In [2]:
import json
import pandas as pd
from datetime import datetime
import glob

In [3]:
%run "DAL.ipynb"

In [4]:
def abbreviate(string):
    words = string.split()
    abbr = ''.join(word[0].upper() for word in words)
    return abbr

In [5]:
class CricketFileProcessor:
    def __init__(self):

        self.meta = None
        self.info = None
        self.innings = None

        self.event = None
        self.event_name = ""
        self.eventID = ""

        self.venue = None
        self.venueID = ""

        self.dates = None
        self.mdate = ""
        self.teams = None
        self.team1ID = ""
        self.team2ID = ""
        self.matchID = ""

        self.peopleDF = pd.DataFrame()
        self.eventDF = pd.DataFrame()

    # initialize IDs 
    def InitializeIDs(self):
        self.event = self.info["event"]
        self.event_name = abbreviate(self.event["name"])
        self.eventID = self.event_name + self.info["season"]

        self.venue = self.info["venue"]
        self.vanueID = abbreviate(self.venue)

        self.dates = self.info["dates"]
        self.mdate = datetime.strptime(self.dates[0], "%Y-%m-%d")
        self.teams = self.info["teams"]
        self.team1ID = abbreviate(self.teams[0])
        self.team2ID = abbreviate(self.teams[1])
        self.matchID = self.event_name + self.mdate.strftime("%Y%m%d")+ self.team1ID + self.team2ID


        # people DF 
    def calculatePeopleDF(self):
        people = self.info["registry"]["people"] 
        people_list = [] 
        for key, value in people.items():
            p = {"ID": value, "name": key}
            people_list.append(p)
        self.peopleDF = pd.DataFrame(people_list)

    def getPeopleID(self, name):
        row = self.peopleDF.loc[self.peopleDF['name'] == name]
        return row.iloc[0]['ID']

    def savePeopleDFinSQL(self):
        for row in self.peopleDF.iterrows():
            SQL = f"EXEC [dbo].[AddUpdatePeople] '{row[1]["ID"]}', '{row[1]["name"]}'"
            DAL.getInstance().execute_query(SQL)

    # Event DF 
    def calculateEventDF(self):
        event_list = {
            
            'eventID': self.eventID, 
            'event_name': self.event["name"],
            'season': self.info["season"]
        }
        self.eventDF = pd.DataFrame(event_list, index=[0])
    def saveEventDFinSQL(self):
        for row in self.eventDF.iterrows():
            SQL = f"EXEC [dbo].[AddUpdateEvent] '{row[1]["eventID"]}', '{row[1]["event_name"]}', '{row[1]["season"]}'"
            DAL.getInstance().execute_query(SQL)

    def calculateVenueDF(self):
        venue_list = {
            'venueID': self.venueID,
            'venue': self.venue
        }
        self.venueDF = pd.DataFrame(venue_list, index=[0])
    def saveVenueDFinSQL(self):
        for row in self.venueDF.iterrows():
            SQL = f"EXEC [dbo].[AddUpdateVenue] '{row[1]["venueID"]}', '{row[1]["venue"]}'"
            DAL.getInstance().execute_query(SQL)

    def calculateTeamDF(self):
        team_list = {
            'teamID': [self.team1ID, self.team2ID], 
            'team': [self.teams[0], self.teams[1]], 
            'gender': [self.info["gender"], self.info["gender"]], 
            'team_type': [self.info["team_type"], self.info["team_type"]]
        }
        self.teamDF = pd.DataFrame(team_list, index=[0,1])
    def saveTeamDFinSQL(self):
        for row in self.teamDF.iterrows():
            SQL = f"EXEC [dbo].[AddUpdateTeams] '{row[1]["teamID"]}', '{row[1]["team"]}', '{row[1]["gender"]}', '{row[1]["team_type"]}'"
            DAL.getInstance().execute_query(SQL)

    def calculateMatchPlayersDF(self):
        playersteam1 = self.info["players"][self.teams[0]]
        playersteam2 = self.info["players"][self.teams[1]]
        player_list = []

        for player in playersteam1:
            playerID = self.getPeopleID(player)
            p = {"matchID": self.matchID, "teamID": self.team1ID, "playerID": playerID}
            player_list.append(p)
        for player in playersteam2:
            playerID = self.getPeopleID(player)
            p = {"matchID": self.matchID, "teamID": self.team2ID, "playerID": playerID}
            player_list.append(p)
        self.playerDF = pd.DataFrame(player_list)
    def savePlayerDFinSQL(self):
        for row in self.playerDF.iterrows():
            SQL = f"EXEC [dbo].[AddUpdateMatchPlayers] '{row[1]["matchID"]}', '{row[1]["teamID"]}', '{row[1]["playerID"]}'"
            DAL.getInstance().execute_query(SQL)
        
    def calculateInningsDF(self):
        inning_list = []
        over_list= []
        inningID = 0 

        for inning in self.innings:
            inningID += 1
            teamID = abbreviate(inning["team"])
            i = {"matchID": self.matchID, "inning": inningID, "team": teamID}
            inning_list.append(i)
            overs = inning["overs"]
            for over in overs:
                over_number = over["over"]
                deliveries = over["deliveries"]
                ball = 0 
                for delivery in deliveries:
                    ball += 1
                    batter = self.getPeopleID(delivery["batter"])
                    bowler = self.getPeopleID(delivery["bowler"])
                    non_striker = self.getPeopleID(delivery["non_striker"])
                    extra_type = "" 
                    extra_runs = 0 
                    if 'extras' in delivery:
                        for key, value in delivery["extras"].items():
                            extra_type = key 
                            extra_runs = value
                    batter_run = 0
                    total_run = 0
                    if 'runs' in delivery:
                        batter_run = delivery["runs"]["batter"]
                        total_run = delivery["runs"]["total"]
                    wicket_kind = "NONE"
                    player_out = "NONE"
                    fielder = "NONE"
                    if 'wickets' in delivery:
                        wicket_kind = delivery["wickets"][0]["kind"]
                        player_out = self.getPeopleID(delivery["wickets"][0]["player_out"])
                        if 'fielders' in delivery:
                            fielder = self.getPeopleID(delivery["wickets"][0]["fielders"][0]["name"])

                    b = {"matchID": self.matchID, "inning": str(inningID), "over": str(over_number), "ball": ball, "batter": batter, "bowler": bowler, 
                        "non_striker": non_striker, "batter_run": str(batter_run), "extra_run": str(extra_runs), "extra_type": extra_type, "total_run": str(total_run),
                        "wicket_kind": wicket_kind, "player_out": player_out, "fielder": fielder}
                    over_list.append(b)
        self.inningDF = pd.DataFrame(inning_list)
        self.overDF = pd.DataFrame(over_list)
    def saveInningDFinSQL(self):
        for row in self.inningDF.iterrows():
            SQL = f"EXEC [dbo].[AddUpdateInning] '{row[1]["matchID"]}', '{row[1]["inning"]}', '{row[1]["team"]}'"
            DAL.getInstance().execute_query(SQL)
    def saveOverDFinSQL(self):
        for row in self.overDF.iterrows():
            SQL = f"EXEC [dbo].[AddUpdateOver] '{row[1]["matchID"]}', '{row[1]["inning"]}', '{row[1]["over"]}', '{row[1]["ball"]}', '{row[1]["batter"]}', '{row[1]["bowler"]}', '{row[1]["non_striker"]}', '{row[1]["batter_run"]}', '{row[1]["extra_run"]}', '{row[1]["extra_type"]}', '{row[1]["total_run"]}', '{row[1]["wicket_kind"]}', '{row[1]["player_out"]}', '{row[1]["fielder"]}'"
            DAL.getInstance().execute_query(SQL)

    def calculateMatchDF(self):
        city = "" 
        if 'city' in self.info:
            city = self.info["city"]

        match_number = "" 
        if 'match_number' in self.info["event"]:
            match_number = self.info["event"]["match_number"]
        stage = "" 
        if 'stage' in self.info["event"]:
            stage = self.info["event"]["stage"]

        player_of_match = ""
        if 'player_of_match' in self.info: 
            player_of_match = self.getPeopleID(self.info["player_of_match"][0])

        toss = self.info["toss"]
        outcome = self.info["outcome"] 
        winner = ""
        loser = "" 
        result = "" 
        by = ""
        if 'result' in outcome: 
            result = outcome["result"]
        if 'winner' in outcome: 
            winner = abbreviate( outcome["winner"])
            loser = self.team1ID if winner == self.team2ID else self.team2ID
            result = outcome["winner"] + " wins"
            dby = outcome["by"]
            if "wickets" in dby:
                by = str(dby["wickets"]) + " wickets"
            elif 'runs' in dby:
                by = str(dby["runs"]) + " runs"



        reserve_umpire = ""
        match_referee = "" 
        tv_umpire = "" 
        umpire1 = "" 
        umpire2 = "" 

        officials = self.info["officials"]
        if 'reserve_umpires' in officials:
            reserve_umpires = officials["reserve_umpires"]
            reserve_umpire = self.getPeopleID(reserve_umpires[0])
        if 'match_referees' in officials:
            match_referees = officials["match_referees"]
            match_referee = self.getPeopleID( match_referees[0])
        if 'tv_umpires' in officials:
            tv_umpires = officials["tv_umpires"]
            tv_umpire = self.getPeopleID(tv_umpires[0])
        if 'umpires' in officials:
            umpires = officials["umpires"]
            umpire1 = self.getPeopleID(umpires[0])
            umpire2 = self.getPeopleID(umpires[1])


        match = {
                'matchID': self.matchID,
                'eventID': self.eventID, 
                'venueID': self.vanueID,
                'meta_version': self.meta["data_version"],
                'meta_created': self.meta["created"], 
                'revision': self.meta["revision"],
                'balls_per_over': self.info["balls_per_over"],
                'match_number': match_number, 
                'stage': stage,
                'city': city, 
                'start_date': self.dates[0], 
                'gender': self.info["gender"],
                'match_type': self.info["match_type"],
                'team1': self.team1ID, 
                'team2': self.team2ID, 
                'toss_winner' : abbreviate( toss["winner"]), 
                'toss_decision': toss["decision"], 
                'overs': self.info["overs"],
                'result': result, 
                'winner': winner, 
                'loser': loser, 
                'win_by': by,
                'player_of_match': player_of_match,
                'reserve_umpire':  reserve_umpire,
                'match_referee': match_referee, 
                'tv_umpire': tv_umpire, 
                'umpire1': umpire1, 
                'umpire2': umpire2,
                'team_type': self.info["team_type"]
                }

        # Create DataFrame
        self.matchDF = pd.DataFrame(match, index=[0])
    def saveMatchDFinSQL(self):
        for row in self.matchDF.iterrows():
            SQL = f"EXEC [dbo].[AddUpdateMatch] '{row[1]["matchID"]}', '{row[1]["eventID"]}', '{row[1]["venueID"]}', '{row[1]["meta_version"]}','{row[1]["meta_created"]}', '{row[1]["revision"]}', '{row[1]["balls_per_over"]}', '{row[1]["match_number"]}', '{row[1]["stage"]}', '{row[1]["city"]}','{row[1]["start_date"]}', '{row[1]["gender"]}', '{row[1]["match_type"]}','{row[1]["team1"]}', '{row[1]["team2"]}', '{row[1]["toss_winner"]}', '{row[1]["toss_decision"]}','{row[1]["overs"]}','{row[1]["result"]}','{row[1]["winner"]}','{row[1]["win_by"]}', '{row[1]["loser"]}', '{row[1]["player_of_match"]}', '{row[1]["reserve_umpire"]}','{row[1]["match_referee"]}','{row[1]["tv_umpire"]}','{row[1]["umpire1"]}','{row[1]["umpire2"]}','{row[1]["team_type"]}'"
            DAL.getInstance().execute_query(SQL)


    def processCricketFile(self, filename):
        # Open the JSON file
        #with open('../PSL/match.json', 'r') as f:
        with open(filename, 'r') as f:
            # Load the JSON data into a Python dictionary
            self.data = json.load(f)

        self.meta = self.data["meta"]
        self.info = self.data["info"]
        self.innings = self.data["innings"]

        self.InitializeIDs()
        self.calculatePeopleDF()
        self.calculateEventDF()
        self.calculateVenueDF()
        self.calculateTeamDF()
        self.calculateMatchPlayersDF()
        self.calculateInningsDF()
        self.calculateMatchDF() 

        self.savePeopleDFinSQL()
        self.saveEventDFinSQL()
        self.saveVenueDFinSQL()
        self.saveTeamDFinSQL()
        self.savePlayerDFinSQL()
        self.saveInningDFinSQL()
        self.saveOverDFinSQL()
        self.saveMatchDFinSQL()

        print(f"Save match file ({filename})... {self.matchID}")


In [6]:
def listAllCricketFilePaths():
    file_paths = glob.glob('C:\\work\\cricket\\PSL\\*')
    PATHS = []
    for file_path in file_paths:
        if '.json' not in file_path:
            file_paths_inner = glob.glob(file_path + '\\*')
            #print(file_paths_inner)
            for file_path_inner in file_paths_inner:
                if '.json' in str(file_path_inner):
                    PATHS.append(str(file_path_inner))
        else:
            if '.json' in str(file_path):
                PATHS.append(str(file_path))
    return PATHS
        


In [7]:
def processAllCricketFiles():
    file_paths = listAllCricketFilePaths()

    cfp = CricketFileProcessor() 
    for file in file_paths:
        cfp.processCricketFile(file)
        #break

In [ ]:
processAllCricketFiles()